In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
from torch.distributions.bernoulli import Bernoulli
from torch.distributions.categorical import Categorical
from torch.distributions.mixture_same_family import MixtureSameFamily
import pandas as pd
import numpy as np
import scipy.stats as ss
import missingno as mno
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, auc, roc_curve
from copy import copy, deepcopy
import zipfile
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
'''torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True'''

Mounted at /content/drive


'torch.cuda.set_device(0)\ntorch.backends.cudnn.benchmark = True'

In [2]:
def categorical_encoding(train, valid, test, categorical):
    new_cat, OHEncoders = [], {}
    for cat in categorical:
        LE = LabelEncoder()
        train[cat] = LE.fit_transform(train[cat])
        valid[cat] = LE.transform(valid[cat])
        test[cat] = LE.transform(test[cat])
        OHEncoders[cat] = LE
    return train, valid, test, OHEncoders


def numerical_scaling(train, valid, test, numerical):
    MS = MinMaxScaler(feature_range=(0, 1))
    scaled_train = MS.fit_transform(train[numerical])
    scaled_valid = MS.transform(valid[numerical])
    scaled_test = MS.transform(test[numerical])
    train[numerical] = scaled_train
    valid[numerical] = scaled_valid
    test[numerical] = scaled_test
    return train, valid, test, MS


def output_scaling(train, valid, test, output_col):
    YScaler = MinMaxScaler(feature_range=(0.5, 1.5))
    Y_train = YScaler.fit_transform(train[[output_col]]).ravel() + 1e-15
    Y_valid = YScaler.transform(valid[[output_col]]).ravel() + 1e-15
    Y_test = test[output_col]
    return Y_train, Y_valid, Y_test, YScaler


def final_features(X_train, categorical, numerical, binary):
    features = categorical + numerical + binary
    unique_col = [col for col in features if len(pd.unique(X_train[col])) == 1]
    return [col for col in features if col not in unique_col]

In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/classification/hotel_cancellation/cleaned_hotel_bookings.csv')
df

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,...,required_car_parking_spaces,total_of_special_requests,Date_sin_dayofweek,Date_cos_dayofweek,Date_sin_dayofyear,Date_cos_dayofyear,reservation_status_date_sin_dayofweek,reservation_status_date_cos_dayofweek,reservation_status_date_sin_dayofyear,reservation_status_date_cos_dayofyear
0,Resort Hotel,0,342,27,0,0,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,8.660254e-01,-0.5,1.716633e-02,-0.999853
1,Resort Hotel,0,737,27,0,0,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,8.660254e-01,-0.5,1.716633e-02,-0.999853
2,Resort Hotel,0,7,27,0,1,1,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,1.224647e-16,-1.0,1.224647e-16,-1.000000
3,Resort Hotel,0,13,27,0,1,1,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,1.224647e-16,-1.0,1.224647e-16,-1.000000
4,Resort Hotel,0,14,27,0,2,2,0.0,0,BB,...,0,1,8.660254e-01,-0.5,0.017166,-0.999853,-8.660254e-01,-0.5,-1.716633e-02,-0.999853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119201,City Hotel,0,23,35,2,5,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,-0.848351,-0.529434,8.660254e-01,-0.5,-9.057023e-01,-0.423914
119202,City Hotel,0,102,35,2,5,3,0.0,0,BB,...,0,2,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304
119203,City Hotel,0,34,35,2,5,2,0.0,0,BB,...,0,4,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304
119204,City Hotel,0,109,35,2,5,2,0.0,0,BB,...,0,0,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304


In [4]:
pd.DataFrame({'Dtype': df.dtypes, 'Nunique': df.nunique(), 'Isnull': df.isnull().sum()}, index=df.columns)

,Dtype,Nunique,Isnull
hotel,object,2,0
is_canceled,int64,2,0
lead_time,int64,479,0
arrival_date_week_number,int64,53,0
stays_in_weekend_nights,int64,17,0
stays_in_week_nights,int64,33,0
adults,int64,14,0
children,float64,5,0
babies,int64,5,0
meal,object,4,0


In [5]:
out_column = 'is_canceled'
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
fig = go.Figure(data=go.Heatmap(z=df[numerical].corr(),x=numerical,y=numerical))
fig.show()

In [6]:
from sklearn.model_selection import train_test_split

index_train, index_test = train_test_split(range(df.shape[0]), test_size=0.2, random_state=42, stratify=df[out_column])
tmp_data_train = df.loc[index_train].reset_index( drop = True )
data_test  = df.loc[index_test].reset_index( drop = True )

index_train, index_valid = train_test_split(tmp_data_train.index, test_size=0.1, random_state=0, stratify=tmp_data_train[out_column])
data_train = tmp_data_train.loc[index_train].reset_index( drop = True )
data_valid = tmp_data_train.loc[index_valid].reset_index( drop = True )

# Creating the X, T and E inputs
X_train, X_valid, X_test = data_train[features], data_valid[features], data_test[features]
Y_train, Y_valid, Y_test = data_train[out_column], data_valid[out_column], data_test[out_column]
pos_weight = len(Y_train) / (Y_train.nunique() * np.bincount(Y_train))

cat_enc_d = {}
for cat in categorical+binary:
    print(cat)
    LE = LabelEncoder()
    X_train[cat] = LE.fit_transform(X_train[cat])
    X_valid[cat] = LE.transform(X_valid[cat])
    X_test[cat] = LE.transform(X_test[cat])
    cat_enc_d[cat] = LE
 
MS = MinMaxScaler(feature_range=(0, 1))
scaled_train = MS.fit_transform(X_train[numerical])
scaled_valid = MS.transform(X_valid[numerical])
scaled_test = MS.transform(X_test[numerical])
X_train[numerical], X_valid[numerical], X_test[numerical] = scaled_train, scaled_valid, scaled_test

X_train.shape, X_valid.shape, X_test.shape

meal
market_segment
distribution_channel
reserved_room_type
deposit_type
customer_type
hotel
is_repeated_guest


<ipython-input-6-0bb2225bf95d>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0bb2225bf95d>:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0bb2225bf95d>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0bb2225b

((85827, 28), (9537, 28), (23842, 28))

In [ ]:
Y_train

0        0
1        0
2        0
3        0
4        1
        ..
91187    0
91188    0
91189    0
91190    0
91191    0
Name: is_canceled, Length: 91192, dtype: int64

In [7]:
from torch import autograd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ClsLoader(Dataset):
    def __init__(self, X, Y, numerical_col, categorical_col):
        self.X1, self.X2, self.Y = X[numerical_col].values.astype(np.float32), X[categorical_col].values, Y.astype(np.float32)
 
    def __len__(self):
        return len(self.Y)
 
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.Y[idx]

class Embedder(nn.Module):
    def __init__(self, embedding_sizes):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])

    def forward(self, x):
        x = [e(x[:,i]) for i,e in enumerate(self.embeddings)]
        return torch.cat(x, 1)

class NNModel(nn.Module):
    def __init__(self, input_shape, units=None, factors=None, activ=True, norm=False, dropout=False, slops=None):
        super().__init__()
        self.input_shape = input_shape
        self.units = units
        self.factors = factors
        self.activ, self.norm = activ, norm
        self.network = nn.ModuleList()
        if self.factors:
            self.units = np.round(self.input_shape * np.asarray(self.factors)).astype(int)
        if self.units is not None:
            self.dropout = np.zeros_like(self.units) if not dropout else dropout
            self.slops = np.full(len(self.units), 1) if slops is None else slops
            for i, j, k in zip(self.units, self.dropout, self.slops):
                if i >= 1:
                    block = self.__build_block__(input_shape, i, p=j, slop=k)
                    self.network.extend(block)
                    input_shape = i
        self.output_shape = input_shape
        self.reset_parameters()
    
    def __build_block__(self, input_shape, units, p, slop):
        block = []
        block.append(nn.Linear(input_shape, units, bias=not self.norm))
        if self.norm:
            block.append(nn.BatchNorm1d(units))
            #block.append(nn.LayerNorm(units, eps=1e-5))
        if self.activ:
            #block.append(nn.LeakyReLU())
            block.append(nn.ELU(slop))
            #block.append(nn.GELU())
        if p > 0:
            block.append(nn.Dropout(p))
        return block
 
    def forward(self, x):
        for layer in self.network:
          tmp = layer(x)
          x = tmp
        return x
 
    def reset_parameters(self):
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                layer.bias.data.fill_(0.1)
 
 
class TabMLP(nn.Module):
    def __init__(self, vocab_size, embed_size, numerical, mlp_units, mlp_dropout=0.00001):
        """
        categories: tuple containing the number of unique values within each category
        """
        super().__init__()
        embedding_sizes = list(zip(vocab_size, embed_size))
        self.cat_embedder = Embedder(embedding_sizes)
        self.norm = nn.BatchNorm1d(numerical, eps=1e-5)
        input_size = numerical + sum(embed_size)
        self.mlp = NNModel(int(input_size), units=mlp_units, factors=None, dropout=[mlp_dropout]*len(mlp_units))
        self.output_shape = mlp_units[-1]

    def forward(self, x_cont, x_cat):
        x = x_cont
        if x_cat.nelement() != 0: #skipped if there's no categorical feature
            x_ = self.cat_embedder(x_cat)
            x = self.norm(x)
            x = torch.cat([x_, x], 1)
        return self.mlp(x)

class BinClsN(nn.Module):
    def __init__(self, shared):
        super(BinClsN, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.shared = shared
        self.pi = nn.Linear(self.shared.output_shape, 1)
 
    def forward(self, x_cont, x_cat):
        z = self.shared(x_cont, x_cat)
        pred = self.pi(z)
        return pred

class MBNModel(nn.Module):#Mixture Binomial Network
    def __init__(self, shared, clf_nn, rt_nn, n_comp):
        super(MBNModel, self).__init__()
        self.shared = shared
        self.clf_nn = clf_nn
        self.rt_nn = rt_nn
        self.n_comp = n_comp
        self.pi = nn.Linear(self.clf_nn.output_shape, self.n_comp) if self.n_comp > 1 else None
        self.ai = nn.Linear(self.rt_nn.output_shape, self.n_comp)
 
    def forward(self, x_cont, x_cat):
        x = self.shared(x_cont, x_cat)
        proba = self.proba_model(x) if self.n_comp > 1 else torch.ones((len(x),1)).to(device)
        rt = self.rt_model(x)
        return proba, rt
 
    def proba_model(self, x):
        model = self.clf_nn(x)
        model = self.pi(model)
        model = nn.Softmax(dim=-1)(model)
        return model
 
    def rt_model(self, x):
        model = self.rt_nn(x)
        model = self.ai(model)
        model = nn.Softmax(dim=-1)(model)
        return model

class BaseParametric:
    def __init__(self, model, numerical_col, categorical_col, resume=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.losses = {'Epoch': [], 'Train': [], 'Test': [], 'BState': [], 'LState': [], 'LR': []}
        self.numerical_col, self.categorical_col = numerical_col, categorical_col
 
    def train_model(self, optim, train_loader, grad_clip, l2_reg):
          total_loss = 0
          self.model = self.model.train()
        #with autograd.detect_anomaly():
          for i, (X1, X2, Y) in enumerate(train_loader):
              X1, X2, Y = X1.to(self.device), X2.to(self.device), Y.to(self.device)
              #self.model.get_weight()
              optim.zero_grad()
              loss = self.loss_function(X1, X2, Y, l2_reg)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              optim.step()
              '''self.model.get_grad()
              print('_'*50)'''
              total_loss += loss.item()
          return total_loss/(i+1)
        
 
    def eval_model(self, test_loader):
        self.model = self.model.eval()
        total_loss = 0
        for i, (X1, X2, Y) in enumerate(test_loader):
            X1, X2, Y = X1.to(self.device), X2.to(self.device), Y.to(self.device)
            loss = self.loss_function(X1, X2, Y, l2_reg=0)
            total_loss += loss.item()
        return total_loss/(i+1)#np.abs(-100. - total_loss)
 
 
    def fit(self, X_train, Y_train, epoch, lr, opt_kwarg, batch_size=None,  grad_clip=100, momentum=0.9,
            X_test=None, Y_test=None, l2_reg=0, eval=True, verbose=True, save=True):
        
 
        batch_size = len(X_train) if batch_size is None else batch_size
        #X_train_uncens, Y_train_uncens, T_train_uncens, X_train_cens, Y_train_cens, T_train_cens = self.process_data(X_train, T_train, E_train)
        train_load = DataLoader(ClsLoader(X_train, Y_train, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=True)  # DATALOADER obj
        if X_test is not None:
            #X_test_uncens, Y_test_uncens, T_test_uncens, X_test_cens, Y_test_cens, T_test_cens = self.process_data(X_test, T_test, E_test)
            test_load = DataLoader(ClsLoader(X_test, Y_test, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=True)  # DATALOADER obj
 
        best_loss = 1e100
        #optim = opt.Adam(self.model.parameters(), lr=lr)
        optim = opt.SGD(self.model.parameters(), lr=lr, momentum=momentum, nesterov=True)

        #scheduler = None
        scheduler = opt.lr_scheduler.CyclicLR(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.ReduceLROnPlateau(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.MultiStepLR(optim, milestones=[28, 120], gamma=0.1)

        eval_score = ''
        for i in range(epoch):
            if verbose:
                print('##### EPOCH ' + str(i) + ' #####')
               
            train_loss = self.train_model(optim, train_load, grad_clip, l2_reg)
            self.losses['LState'] = deepcopy(self.model.state_dict())
    
            if verbose:
                print('train loss : ', train_loss)
            self.losses['Epoch'].append(i), self.losses['Train'].append(train_loss)
    
            if X_test is not None:
                valid_loss = self.eval_model(test_load)

                if verbose:
                    print('test loss : ', valid_loss)
                self.losses['Test'].append(valid_loss)
    
                if scheduler is not None:
                    '''scheduler.step(valid_loss)
                    self.losses['LR'].append(optim.param_groups[0]['lr'])'''
                    scheduler.step()
                    self.losses['LR'].append(scheduler.get_last_lr()[0])
    
                if valid_loss < best_loss:
                    self.losses['BState'] = deepcopy(self.model.state_dict())
                    best_loss = valid_loss
                    print('===========SAVE===========')
            

    def feature_importance(self, rep, X_test, Y_test, batch_size=None):
        res = np.zeros((rep, X_test.shape[1]))
        batch_size = len(X_train) if batch_size is None else batch_size
        test_load = DataLoader(ClsLoader(X_test, Y_test, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=True)  # DATALOADER obj
        base_loss = self.eval_model(test_load)
        origin = X_test.copy()
        for i, col in enumerate(X_test.columns):
            for j in range(rep):
                X_test.loc[:,col] = np.random.permutation(X_test.loc[:, col])
                test_load = DataLoader(ClsLoader(X_test, Y_test, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=True)  # DATALOADER obj
                loss = self.eval_model(test_load)
                res[j, i] = base_loss - loss
                X_test = origin
        res = np.abs(res)
        return {'importances': res, 'importances_mean': np.mean(res, 0), 'importances_std': np.std(res, 0)}

class BCEModel(BaseParametric):
    def __init__(self, model, numerical_col, categorical_col,resume=None):
        super(BCEModel, self).__init__(model, numerical_col, categorical_col,)
        self.pos_weight = torch.from_numpy(pos_weight[[0]]).to(device)

    def loss_function(self, X1, X2, Y, l2_reg):
        pred = self.model(X1, X2)
        bce_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        loss = bce_loss(pred, Y)
        return loss

    def prdict(self, X):
        self.model.eval()
        X1 = torch.tensor(X[self.numerical_col].values.astype(np.float32)).to(self.device)
        X2 = torch.tensor(X[self.categorical_col].values).to(self.device)
        pred = nn.Sigmoid()(self.model(X1, X2))
        return pred

class MixtureBinomialModel(BaseParametric):
    def __init__(self, model, numerical_col, categorical_col,resume=None):
        super(MixtureBinomialModel, self).__init__(model, numerical_col, categorical_col,)

    def loss_function(self, X1, X2, Y, l2_reg):
        pi, rt = self.model(X1, X2)
        mix = Categorical(pi)
        comp = Bernoulli(rt, validate_args=None)
        pdf = MixtureSameFamily(mix, comp).log_prob(Y)
        ll1 = -torch.mean(pdf)
        return ll1

    def predict(self, X):
        self.model.eval()
        X1 = torch.tensor(X[self.numerical_col].values.astype(np.float32)).to(self.device)
        X2 = torch.tensor(X[self.categorical_col].values).to(self.device)
        pi, rt = self.model(X1, X2)
        mix = Categorical(pi)
        comp = Bernoulli(rt, validate_args=None)
        msf = MixtureSameFamily(mix, comp)
        pred = msf.mean
        return pred, pi, rt
 
def gradient_clipper(model: nn.Module, val: float) -> nn.Module:
    def process_grad(grad):
        grad[grad != grad] = 1e-10
        return torch.clamp(grad, -val, val)
    for parameter in model.parameters():
        parameter.register_hook(lambda grad: process_grad(grad))
    
    return model

In [8]:
epoch, lr, batch_size, d, mlp_d = 50000, 1e-4, 4096, 0.000001, 1e-6
cyclic_kwarg = {'base_lr': lr, 'max_lr': 1e-2, 'step_size_up':200, 'step_size_down':200}
plateau_kwarg = {'factor':0.5, 'patience':200, 'verbose':True, 'min_lr':1e-7, 'mode':'min'}
embed_size = np.ceil(np.sqrt(list(X_train[categorical+binary].nunique()))).astype(int)

shared_nn = TabMLP(df[categorical+binary].nunique(), embed_size, len(numerical), [128, 512, 768, 512], mlp_dropout=mlp_d)

nn_model = gradient_clipper(BinClsN(shared_nn), 10)
#nn_model.load_state_dict(best_state)
print(nn_model)
print(sum(p.numel() for p in nn_model.parameters() if p.requires_grad))
dws = BCEModel(nn_model, numerical, categorical+binary)
dws.fit(X_train, Y_train.values[:,None], epoch, lr, cyclic_kwarg, batch_size=batch_size, grad_clip=10, momentum=0.9,
        X_test=X_valid, Y_test=Y_valid.values[:,None], l2_reg=0, eval=False, verbose=True)

Streaming output truncated to the last 5000 lines.
===========SAVE===========
##### EPOCH 41 #####
train loss :  0.3638188356444949
test loss :  0.3738648096720378
##### EPOCH 42 #####
train loss :  0.3624228537082672
test loss :  0.3690970838069916
===========SAVE===========
##### EPOCH 43 #####
train loss :  0.36082471410433453
test loss :  0.3656560182571411
===========SAVE===========
##### EPOCH 44 #####
train loss :  0.35909906597364516
test loss :  0.36760156353314716
##### EPOCH 45 #####
train loss :  0.35772750633103506
test loss :  0.3634152412414551
===========SAVE===========
##### EPOCH 46 #####
train loss :  0.35615817705790204
test loss :  0.36941473682721454
##### EPOCH 47 #####
train loss :  0.3546707204410008
test loss :  0.36047206322352093
===========SAVE===========
##### EPOCH 48 #####
train loss :  0.3534911416825794
test loss :  0.36488280693689984
##### EPOCH 49 #####
train loss :  0.3518387419836862
test loss :  0.3612990379333496
##### EPOCH 50 #####
train loss 

KeyboardInterrupt: ignored

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

best_state = deepcopy(dws.losses['BState'])
dws.model.load_state_dict(best_state)
print(np.min(dws.losses['Test']))

fig = make_subplots(rows=3, cols=1)
s = 0
model = dws
fig.append_trace(go.Scatter(x=model.losses['Epoch'][s:], y=model.losses['Train'][s:],mode='lines',name='Train'), row=1, col=1)
fig.append_trace(go.Scatter(x=model.losses['Epoch'][s:], y=model.losses['Test'][s:],mode='lines',name='Test'), row=2, col=1)
fig.append_trace(go.Scatter(x=model.losses['Epoch'][s:], y=model.losses['LR'][s:],mode='lines',name='LR'), row=3, col=1)
fig.update_layout(height=1000, width=1500, title_text="Stacked Subplots")
fig.show()

0.006716994879146417


In [10]:
pred = []
for i in range(0, len(X_valid), 1000):
    pred_ = dws.prdict(X_valid.iloc[i:i+1000])
    pred_ = pred_.cpu().data.numpy().ravel()
    pred.extend(pred_)
pred = np.asarray(pred).ravel()

auc = roc_auc_score(Y_valid, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_valid, np.round(pred)), 'PRE': precision_score(Y_valid, np.round(pred)), 'REC': recall_score(Y_valid, np.round(pred)), 'F1':f1_score(Y_valid, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.999752,0.997903,0.998299,0.996041,0.997169


In [11]:
pred = []
for i in range(0, len(X_test), 1000):
    pred_ = dws.prdict(X_test.iloc[i:i+1000])
    pred_ = pred_.cpu().data.numpy().ravel()
    pred.extend(pred_)
pred = np.asarray(pred).ravel()

auc = roc_auc_score(Y_test, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_test, np.round(pred)), 'PRE': precision_score(Y_test, np.round(pred)), 'REC': recall_score(Y_test, np.round(pred)), 'F1':f1_score(Y_test, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.999843,0.997567,0.998637,0.994796,0.996713


In [12]:
confusion_matrix(Y_test, np.round(pred))

array([[14991,    12],
       [   46,  8793]])

In [13]:
imp = dws.feature_importance(10, X_test, Y_test.values[:,None], batch_size=None)
fig = go.Figure()
for i in range(X_test.shape[1]):
    fig.add_trace(go.Box(x=imp['importances'][:, i], name=X_test.columns[i]))
fig.show()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

